# NeuroDataReHack 2022 - analysis and work flow

-----

- Short term goal: Apply coherence detection on pre-existing dataset (Chung et al )
- Long term goal: test whether synchrony is predictive of anatomy, or the other way around.

*Make sure to set kernel to ipykernel*


----

Import some libraries

In [1]:
# import vars and dandi api
import pynwb
import requests
import numpy as np
from dandi.dandiapi import DandiAPIClient
from pynwb import NWBHDF5IO
from nwbwidgets import nwb2widget
import os # import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

/opt/conda/lib/python3.10/site-packages/xarray/backends/cfgrib_.py:29: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


Ben Dichter functions

In [ ]:
def _search_assets(url, filepath):
    response = requests.request("GET", url, headers={"Accept": "application/json"}).json() 
    
    for asset in response["results"]:
        if filepath == asset["path"]:
            return asset["asset_id"]
    
    if response.get("next", None):
        return _search_assets(response["next"], filepath)
    
    raise ValueError(f'path {filepath} not found in dandiset {dandiset_id}.')

def get_asset_id(dandiset_id, filepath):
    url = f"https://api.dandiarchive.org/api/dandisets/{dandiset_id}/versions/draft/assets/"
    return _search_assets(url, filepath)

def get_s3_url(dandiset_id, filepath):
    """Get the s3 location for any NWB file on DANDI"""

    asset_id = get_asset_id(dandiset_id, filepath)
    url = f"https://api.dandiarchive.org/api/dandisets/{dandiset_id}/versions/draft/assets/{asset_id}/download/"
    
    s3_url = requests.request(url=url, method='head').url
    if '?' in s3_url:
        return s3_url[:s3_url.index('?')]
    return s3_url

----
Get session data given dandiset ID

In [ ]:
# define dataset
#dandiset_id = '000004' # dataset
dandiset_id = '000065'

# get dataset
with DandiAPIClient() as client:
    # get dandiset using the ID code
    dandiset = client.get_dandiset(dandiset_id,'draft')
    
    # list splits the "zipped" generator file
    assets   = list(dandiset.get_assets())
    
    # list comprehension
    rats = {os.path.split(x.path)[0] for x in assets} # list comprehension, for loop, {} if unique

    # actually just work with sessions for now and call it a day
    sessions = [x.path for x in assets]
    #print(sessions)

    #asset = client.get_dandiset(dandiset_id, 'draft').get_asset_by_path(filepath)
    #s3_url = asset.get_content_url(follow_redirects=1, strip_query=True)

# temporarily (I mean before actual data analysis), get one session to get streaming working
tempAssetID = sessions[0]
asset_id = get_asset_id(dandiset_id, tempAssetID) # this is the URL for download
s3_path  = get_s3_url(dandiset_id, tempAssetID) # this is the path to streaming
#print(asset_id)

# use the "Read Only S3" (ros3) driver to stream data directly from DANDI (or any other S3 location)
io = NWBHDF5IO(s3_path, mode='r', load_namespaces=True, driver='ros3')

nwb = io.read()

# this is the golden function here -- Not really working for some reason
widgetOut = 1
if widgetOut==1:
    nwb2widget(nwb)

#Get the fields within the NWB file
nwbFields = nwb.fields
#print(nwbFields)

If working with dataset 004

In [ ]:
if dandiset_id == '000004':
    ##Plot the Waveforms from the NWB file

    #Which channel_index to plot? 
    channel_index = [0]

    # get Waveform Means from the NWB file
    allwaveformLearn = np.asarray(nwb.units['waveform_mean_encoding'].data)
    allwaveformRecog = np.asarray(nwb.units['waveform_mean_recognition'].data)

    print(nwb.units)

    # Choose Which Channel Index to Plot
    waveformLearn = allwaveformLearn[channel_index, :][0]
    waveformRecog = allwaveformLearn[channel_index, :][0]

    #get brain Areas
    brainAreas = np.asarray(nwb.electrodes['location'].data)


    #Plot the mean waveforms

    fig, axes = plt.subplots(1, 2, figsize = (15, 10)) 

    #Plot Learning
    axes[0].plot(range(len(waveformLearn)), waveformLearn, color = 'blue', marker = 'o', linestyle='dashed',
                linewidth=1, markersize=3)
    axes[0].set_title('Learning, session: {}, brain Area: {}'.format(nwb.identifier, brainAreas[channel_index][0]))
    axes[0].set_xlabel('time (in ms)')
    axes[0].set_ylabel('\u03BCV')

    #Plot Recog
    axes[1].plot(range(len(waveformRecog)), waveformRecog, color = 'green', marker = 'o', linestyle='dashed',
                linewidth=1, markersize=3)
    axes[1].set_title('Recognition, session: {}, brain Area: {}'.format(nwb.identifier, brainAreas[channel_index][0]))
    axes[1].set_xlabel('time (in ms)')
    axes[1].set_ylabel('\u03BCV')